In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn import preprocessing
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
import plotly.graph_objects as go
from sklearn.metrics import r2_score



%matplotlib inline

In [2]:
train = pd.read_csv('/kaggle/input/ventilator-pressure-prediction/train.csv')
print('Train shape: ', train.shape)
test = pd.read_csv('/kaggle/input/ventilator-pressure-prediction/test.csv')
print('Test shape: ', test.shape)
submisison = pd.read_csv('/kaggle/input/ventilator-pressure-prediction/sample_submission.csv')

Train shape:  (6036000, 8)
Test shape:  (4024000, 7)


In [3]:
train.head()

,id,breath_id,R,C,time_step,u_in,u_out,pressure
0,1,1,20,50,0.000000,0.083334,0,5.837492
1,2,1,20,50,0.033652,18.383041,0,5.907794
2,3,1,20,50,0.067514,22.509278,0,7.876254
3,4,1,20,50,0.101542,22.808822,0,11.742872
4,5,1,20,50,0.135756,25.355850,0,12.234987


In [4]:
#''''df = train
#print(max(df['time_step']))
#print(min(df['time_step']))'''

In [5]:
#''''df = train
#print(max(df['breath_id']))
#print(min(df['breath_id']))
#df['breath_id']

In [6]:
test.head()

,id,breath_id,R,C,time_step,u_in,u_out
0,1,0,5,20,0.000000,0.000000,0
1,2,0,5,20,0.031904,7.515046,0
2,3,0,5,20,0.063827,14.651675,0
3,4,0,5,20,0.095751,21.230610,0
4,5,0,5,20,0.127644,26.320956,0


In [7]:
train.dtypes

id             int64
breath_id      int64
R              int64
C              int64
time_step    float64
u_in         float64
u_out          int64
pressure     float64
dtype: object

In [8]:
dataTrn = train.copy()
dataTrn.drop(columns=['id', 'breath_id', 'R', 'C', 'time_step'], axis = 1, inplace = True)

In [9]:
X = dataTrn.drop(columns='pressure')
X = X[:80000]
y = dataTrn['pressure']
y = y[:80000]


In [10]:
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.30, random_state=1)
print('Dimension of X_train:', X_train.shape,'\nDimension of X_validation:', X_validation.shape,'\nDimension of y_train:', y_train.shape,'\nDimension of y_validation:', y_validation.shape)


Dimension of X_train: (56000, 2) 
Dimension of X_validation: (24000, 2) 
Dimension of y_train: (56000,) 
Dimension of y_validation: (24000,)


In [11]:
X_test, X_validation1, y_test, y_validation1 = train_test_split(X, y, test_size=0.50, random_state=1)
print('Dimension of X_test:', X_test.shape,'\nDimension of X_validation:', X_validation1.shape,'\nDimension of y_test:', y_test.shape,'\nDimension of y_validation:', y_validation1.shape)


Dimension of X_test: (40000, 2) 
Dimension of X_validation: (40000, 2) 
Dimension of y_test: (40000,) 
Dimension of y_validation: (40000,)


In [12]:
RF_model=RandomForestRegressor(n_estimators = 501, random_state=0)
RF_model.fit(X_train, y_train)


RandomForestRegressor(n_estimators=501, random_state=0)

In [13]:
RF_model1=RandomForestRegressor(n_estimators = 501, random_state=0)
RF_model1.fit(X_test , y_test)

RandomForestRegressor(n_estimators=501, random_state=0)

In [14]:
RF_train_predict = RF_model.predict(X_train)
RF_validation_predict = RF_model.predict(X_validation)


In [15]:
RF_test_predict = RF_model1.predict(X_test)
RF_validation_predict = RF_model1.predict(X_validation1)

In [16]:
def mape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual)) * 100

In [17]:
RF_MAPE_train = mape(y_train , RF_train_predict)
RF_MAPE_test = mape(y_test , RF_test_predict)

RF_R2_train = r2_score(y_train , RF_train_predict)
RF_R2_test = r2_score(y_test , RF_test_predict)

RF_MAE_train = mean_absolute_error(y_train , RF_train_predict)
RF_MAE_test = mean_absolute_error(y_test , RF_test_predict)

In [18]:
print("train r2_score : " ,RF_R2_train)
print("test r2_score : " ,RF_R2_test)

print("train mape score : " ,RF_MAPE_train)
print("test mape score : " ,RF_MAPE_test)

print("train MAE score : " ,RF_MAE_train)
print("test MAE score : " ,RF_MAE_test)

train r2_score :  0.7303750479105938
test r2_score :  0.7305502153980109
train mape score :  20.901691958115702
test mape score :  20.894285378276578
train MAE score :  2.3193338825653402
test MAE score :  2.3229971689007853


In [19]:
RF_model_score_test = RF_model1.score(X_test , y_test)
RF_model_score_train = RF_model.score(X_train, y_train)

In [20]:
resultRF = pd.DataFrame({'R Squared Score' : RF_R2_train,
                        'MAPE Score' : RF_MAPE_train,
                        'MAE Score' : RF_MAE_train,
                        'Model Score' : RF_model_score_train}
                       ,index = ['Train Data'])

In [21]:
resultRF1 = pd.DataFrame({'R Squared Score' : RF_R2_test,
                        'MAPE Score' : RF_MAPE_test,
                        'MAE Score' : RF_MAE_test,
                        'Model Score' : RF_model_score_test}
                       ,index = ['Test Data'])
result = pd.concat([resultRF , resultRF1])
result

,R Squared Score,MAPE Score,MAE Score,Model Score
Train Data,0.730375,20.901692,2.319334,0.730375
Test Data,0.730550,20.894285,2.322997,0.730550
